# EINmodel for desicion making

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---- Activation function (sigmoid): conversion from synaptic input to firing rate ----
def sigmoid(x, theta=1.0):
    # theta: slope parameter (controls steepness/threshold)
    return 1.0 / (1.0 + np.exp(-x/theta))

# ---- Model parameter settings ----
WEE = 1.6  # recurrent excitatory connection strength (E->E, applies to both E_R and E_L)
WEI = 1.0  # inhibitory-to-excitatory connection (I->E_R, I->E_L)
WIE = 1.0  # excitatory-to-inhibitory connection (E_R, E_L -> I)
WII = 1.2  # inhibitory self-connection (I->I; here set to zero, not used)
IE_R = 1.0 # external input to E_R (e.g., task stimulus)
IE_L = 1.0 # external input to E_L
II = 1.0   # external input to I
tau = 10.0 # time constant (how quickly firing rate can change, usually 10–20)
theta = 1.0  # sigmoid slope

# ---- Steady-state value of inhibitory population (I), given E_R and E_L ----
def I_steady(E_R, E_L):
    # I = sigmoid(WIE*(E_R + E_L) - WII*I + II)
    # Here, I->I is omitted, so I = sigmoid(WIE*(E_R + E_L) + II)
    inp = WIE*(E_R + E_L) - WII*0 + II
    return sigmoid(inp, theta)

# ---- Create state space grid (E_R, E_L axes) ----
E_range = np.linspace(0, 1.5, 25)  # Range of possible E_R, E_L firing rates
E_R_grid, E_L_grid = np.meshgrid(E_range, E_range)  # 2D grid

dER = np.zeros_like(E_R_grid)  # Stores dE_R/dt at each grid point
dEL = np.zeros_like(E_L_grid)  # Stores dE_L/dt at each grid point

# ---- For each grid point, compute the ODE right-hand side (dE/dt) ----
for i in range(E_R_grid.shape[0]):
    for j in range(E_R_grid.shape[1]):
        E_R = E_R_grid[i, j]  # Current E_R
        E_L = E_L_grid[i, j]  # Current E_L
        I = I_steady(E_R, E_L)  # Compute steady-state value of I for these E_R/E_L
        # Total input: recurrent excitation, inhibition, external input
        inp_R = WEE*E_R - WEI*I + IE_R  # Total synaptic input to E_R
        inp_L = WEE*E_L - WEI*I + IE_L  # Total synaptic input to E_L
        # Main ODE: τ dE/dt = -E + f(total input)
        dER[i, j] = (-E_R + sigmoid(inp_R, theta)) / tau  # Time derivative of E_R
        dEL[i, j] = (-E_L + sigmoid(inp_L, theta)) / tau  # Time derivative of E_L

# ---- Plot vector field (phase plot: direction and magnitude of dE/dt at each point) ----
plt.figure(figsize=(7,7))
plt.quiver(E_R_grid, E_L_grid, dER, dEL, angles='xy')
plt.xlabel('E_R firing rate')  # x-axis: E_R firing rate
plt.ylabel('E_L firing rate')  # y-axis: E_L firing rate
plt.title('Phase plot (dE_R/dt, dE_L/dt) with I as steady-state')
plt.grid()
plt.show()